In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import re


In [2]:
a = 10
b = 20
a + b


30

In [2]:
# aioushndiausndaiusndu

In [3]:
# iqbal

In [4]:
# test